In [4]:
import tensorflow as tf
import numpy as np

train_data = np.fromfile("train_data2.bin", dtype=np.uint8).reshape(25000,784)
train_labels = np.fromfile("train_labels2.bin", dtype=np.uint8).reshape(25000,1)

new_labels = list()

def opposite(value):
    if (value==0):
        return 1
    else:
        return 0

for i in range(25000):
    new_labels.append([int(train_labels[i]), opposite(int(train_labels[i]))])

train = train_data[:20000]
tlabels = np.squeeze(train_labels[:20000])

validate = train_data[20000:]
vlabels = np.squeeze(train_labels[20000:])

# The true test data (for estimation)

#test_data = np.fromfile("test_data.bin", dtype=np.uint8).reshape(12500,10000)


In [6]:
import tensorflow as tf

# Start an interactive session

sess = tf.InteractiveSession()


# Define the placeholder variable for inputs and truth labels.
# This will be the input that we provide when we actually run our model

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 2])

In [7]:
# Define variables. Initialized in this case to vectors/matrices of 0's
# These will be modified during the execution of the model

W = tf.Variable(tf.zeros([784,2]))
b = tf.Variable(tf.zeros([2]))

In [8]:
sess.run(tf.initialize_all_variables())

In [9]:
# Define the model output as the softmax resulting from the matrix operation x*W + b

y = tf.nn.softmax(tf.matmul(x, W) + b)

In [10]:
# Define placeholder for correct values
# Any number of rows, 10 columns



# Define cross-entropy function for testing the efficiency of our predictions
# Starts by logging all values in the y vector, then multiplies by the truth (y_)
# It then adds up all the values in the new vector, and takes the average across all
#     observations

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [11]:
# Minimize cross-entropy using a gradient descent, learning rate .5

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [21]:
# Execute training step 1000 times (stochastic training)

for i in range(1000):
    sample = np.random.randint(0,20000,100)
    batch = train[sample]
    batch_y = np.array(new_labels)[sample]
    train_step.run(feed_dict={x: batch, y_: batch_y})

In [22]:
# Check how often we arrive at the correct prediction

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# Calculate the accuracy after casting booleans as float values

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
# Query accuracy

print(accuracy.eval(feed_dict={x: validate, y_: new_labels[20000:]}))

0.4966


# Starting the Convolutional Neural Network

This will help to improve accuracy (hopefully by a lot)

Will be using rectified linear unit neurons (ReLU). Based on activation function $f(x) = max(0,x)$. This function is considered a "Ramp function"

In [25]:
# Initialize weights, do it with small positive noise, to avoid dead neurons, 
#     break symmetry, and prevent 0 gradients

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [26]:
# Define convolutions with a stride size of 1 and with 0-padding, so that output is the same size as input
# Pooling will be max pooling over 2x2 blocks of data

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [27]:
# Create first layer of the neural net: convolution followed by max pooling
# Compute 32 features for each 5x5 patch, with weight tensor of dimension [5, 5, 1, 32]
#     where dimensions are patch size (5 and 5), input channels (1), and output channels (32)
# Create a bias vector with an element for each output channel

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])



In [28]:
# Reshape x to a 4d tensor: first dimension = ?, 
#    second and third: image width and height, fourth: number of color channels

x_image = tf.reshape(x, [-1, 28, 28, 1])

In [29]:
# Convolve x_image with the weight and add the bias, apply the ReLU function, and max pool

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [30]:
# Stack several layers in order to build a deep network. 
# This layer will have 64 features for each 5x5 patch

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [31]:
# Densely Connected Layer. We have a 7x7 image, and use 1024 neurons to allow processing
#     on the entire image.
# Reshape tensor from pooling layer into batch of vectors, multiply by weight matrix, and add bias
#     then apply the ReLU function

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [32]:
# Reduce overfitting by applying dropout, assigning probabilities that a neuron's output is kept
#     so that we can turn on dropout for training, and off for testing

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [35]:
# Add a softmax layer

W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
# ADAM optimizer, include a probability for the dropout rate, log every 100th iteration's accuracy
# Run 20,000 iterations of the neural net on 50 observations (pre-dropout)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

for i in range(20000):
    sample = np.random.randint(0,20000,50)
    batch = train[sample]
    batch_y = np.array(new_labels)[sample]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {
                x:batch, y_: batch_y, keep_prob: 1.0
            })
        print("Step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x:batch, y_:batch_y, keep_prob:0.5})
    
print("Test Accuracy %g" % accuracy.eval(feed_dict={
            x: validate, y_: new_labels[20000:], keep_prob:1.0
        }))